In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# changing matplotlib the default style
matplotlib.style.use('ggplot')

from PDB_Order_Fixer import PDB_Order_Fixer
import mdtraj as md
import os
import numpy as np
import h5py

import datetime
import glob
import copy
from functools import partial 
import operator
import time

import random 
import subprocess
from subprocess import Popen
import sys
from io_functions import *
from custom_clusterer import *
from custom_tica import *
from custom_featurizer import *
from pdb_editing import *
from analysis import *
from io_functions import *
#from topology_fixing import *
from subsampling import *
from conversions import *
from custom_msm import *
from grids import *
from docking_analysis import *

Couldn't import dot_parser, loading of dot files will not be possible.


/home/enf/software/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In this iPython notebook, we will featurize MOR ligand binding simulation by pairwise distances between the ligand and different receptor residues. We will then perform tICA and prospectively build an MSM. 

In [3]:
from detect_intermediates import *
from interpret_tICs import *

we are operating on biox3


In [4]:
from mor_h8_feature_types import *
from get_variable_names import *
from mor_h8_tica_config import *
from residue import Residue, Atom
ori_feature_name = copy.deepcopy(feature_name)

tm6_tm3_residues
[R279, R165]
[65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 273, 274, 275, 276, 277, 278,

In [5]:
rho = 0.1
rho_string = "0pt1"
lag_time = 5
n_components = 10
n_clusters = 1000

rho = 0.01
rho_string = "0pt01"

In [6]:
schemes = ["closest-heavy", "CA"]
feature_name = "%s-CA" %ori_feature_name
#feature_name = "%s_phi_psi_chi2" %feature_name
(active_ref_dir, inactive_ref_dir, simulation_ref_dir, scripts_dir,
          ligand_dir, agonist_dir, inverse_agonist_dir, biased_agonist_dir, ref_receptors_dir, whole_trajectory_pnas,
          sasa_file) = get_base_files(base)

tica_dir = get_tica_dir(base, is_sparse, lag_time, n_components, feature_name, 
                                 wolf_string, shrinkage_string, rho_string)
ori_tica_dir = copy.deepcopy(tica_dir)
#tica_dir = "%s-normalized" % ori_tica_dir
features_dir = get_features_dir(base, feature_name)

landmarks_dir = get_landmarks_dir(tica_dir)
analysis_dir = get_analysis_dir(tica_dir, n_clusters, sampling_method)
gmm_dir = get_gmm_dir(tica_dir)
rf_dir = get_rf_dir(tica_dir)


ref_tica_dir, ref_tica_coords = get_ref_tica_dirs(tica_dir)

graph_file = get_graph_file(tica_dir, msm_lag_time, n_clusters)

pnas_titles =  ["tm6_tm3_dist", "rmsd_npxxy_inactive", "rmsd_npxxy_active", "rmsd_connector_inactive", "rmsd_connector_active"]
pnas_features_dir = analysis_dir


(clusterer_dir, msm_model_dir, macrostate_dir, features_known, model_dir, projected_features_dir,
         projection_operator_dir, ktica_fit_model_filename, ktica_projected_data_filename, nystroem_data_filename,
         mutual_information_csv, pearson_csv) = get_tica_files(base, tica_dir, n_clusters, msm_lag_time, n_macrostates)

(standardized_features_dir, feature_residues_csv, feature_residues_pkl,
          contact_csv, ref_features_dir) = get_feature_files(features_dir)

(kmeans_csv, tica_coords_csv, features_csv, active_rmsd_dir, inactive_rmsd_dir, active_pnas_dir, inactive_pnas_joined, active_pnas_joined,
        clusters_map_file, ktica_clusters_map_file, analysis_file, combined_file, docking_summary, docking_joined, docking_z_scores_csv,
        aggregate_docking, aggregate_docking_joined, docking_pnas_joined, aggregate_docking_pnas, aggregate_docking_pnas_joined, docking_multiple_ligands,
        docking_distances_file, docking_pdf, mmgbsa_docking_distances, pnas_coords, mmgbsa_dir, mmgbsa_csv, mmgbsa_pdf, aggregate_mmgbsa,
        aggregate_mmgbsa_joined, aggregate_mmgbsa_pnas_joined, mmgbsa_z_scores_csv, active_clusters_csv, intermediate_clusters_csv,
        inactive_clusters_csv, pnas_clusters_averages, tica_clusters_averages, tica_classes_csv, tica_samples_csv, subgraph_save_base,
        degree_save_base, degree_map_csv, degree_z_map_csv, aggregate_docking_pnas_degree_z_joined, tic_residue_csv, feature_coefs_csv,
        duplicated_feature_coefs_csv) = get_analysis_files(analysis_dir, n_clusters, tica_dir, tica_dir, sampling_method, n_samples, precision,
                                                           msm_lag_time)

(inactive_pnas_distances_dir, active_pnas_distances_dir, active_pnas_all_distances_dir,
          inactive_pnas_distances_new_csv, active_pnas_distances_new_csv, active_pnas_joined, active_pnas_means, pnas_coords_dir,
          pnas_coords_csv, pnas_all_coords_csv, pnas_coords_hexbin_dir, pnas_coords_co_crystallized_docking_dir,
          pnas_coords_active_colors_dir, user_defined_features_file, reaction_coordinates_trajs_file) = get_pnas_files(whole_trajectory_pnas, pnas_features_dir)

features_dir = get_features_dir(base, feature_name)



graph_file = get_graph_file(tica_dir, msm_lag_time, n_clusters)
(scripts_dir, pymol_fixpdb_dir) = get_script_dir(scripts_dir)
(save_dir, reimaged_dir, mae_dir, combined_reimaged_dir, grid_dir, docking_dir) = get_docking_dirs(tica_dir, n_clusters, n_components, n_samples, sampling_method, precision)


/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA
/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA


In [24]:
from ipyparallel import Client
rc = Client()
print(len(rc.ids))
dview = rc[:]
dview.map(os.chdir, ['/home/enf/b2ar_analysis/conformation']*len(rc.ids))

420


<AsyncMapResult: chdir>

In [9]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *
user_defined_features_dir = "%s/user_defined_features" %traj_dir
if not os.path.exists(user_defined_features_dir):
    os.makedirs(user_defined_features_dir)
compute_user_defined_features_wrapper(traj_dir, traj_ext, inactive_dir, active_dir, None,
                                        feature_name_residues_dict, user_defined_features_file, worker_pool=dview, 
                                        save_dir=user_defined_features_dir, parallel=False)

Saving "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/user_defined_features.h5"... (<type 'list'>)


In [26]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *
suf_trajectories = [t for t in get_trajectory_files(traj_dir, ".h5") if "suf" in t]
fix_chain_names(suf_trajectories, dview)
#featurize_contacts_custom(traj_dir, features_dir = features_dir, traj_ext = traj_ext, contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir], contact_residues=contact_residues,
#                          residues_map = None, contact_cutoff = cutoff, parallel = False, exacycle = exacycle, traj_top_structure = None, iterative=False,
#                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes)

In [27]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *
featurize_contacts_custom(traj_dir, features_dir = features_dir, traj_ext = traj_ext, contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir], contact_residues=contact_residues,
                          residues_map = None, contact_cutoff = cutoff, parallel = True, exacycle = exacycle, traj_top_structure = None, iterative=False,
                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes, dihedral_types=["phi", "psi", "chi2"])

already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already featurized
already feat

LinAlgError: Singular matrix

In [ ]:
fit_normalizer(features_dir)

loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA_phi_psi_chi2/apo_rep_1.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA_phi_psi_chi2/apo_rep_10.datasetlooking at 2220 pairs for trajectorylooking at 2220 pairs for trajectorylooking at 2220 pairs for trajectory


new features /home/enf/md_simulations/MOR/h8_reimaged/trajectories/apo_reseeded_rep_237.h5 has shape: new features /home/enf/md_simulations/MOR/h8_reimaged/trajectories/apo_reseeded_rep_235.h5 has shape: new features /home/enf/md_simulations/MOR/h8_reimaged/trajectories/apo_reseeded_rep_234.h5 has shape: 


(249, 5850)
(244, 5850)
(247, 5850)

loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA_phi_psi_chi2/apo_rep_100.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA_phi_psi_chi2/apo_rep_101.dataset
loading /home/enf/md_si

In [9]:
import pickle
with open(feature_residues_pkl, "rb") as f:
    feature_residues = pickle.load(f)

In [10]:
ref_features_dir

'/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/reference_receptors'

In [7]:
import custom_tica
reload(custom_tica)
from custom_tica import *
import time
fit_and_transform(features_directory = features_dir, model_dir = tica_dir, stride=5, lag_time = lag_time, n_components = n_components, sparse = sparse, wolf = wolf, rho = rho, shrinkage = shrinkage, parallel=True, traj_ext = traj_ext, normalize=True, partial_fit=True)

loading "/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/normalizer.h5"...


IOError: Unable to load /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/normalizer.h5 using the pickle or joblib protocol.
Pickle: [Errno 2] No such file or directory: '/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/normalizer.h5'
Joblib: [Errno 2] No such file or directory: '/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/normalizer.h5'

In [ ]:
featurize_contacts_custom(ref_receptors_dir, features_dir = ref_features_dir, traj_ext = ".pdb", contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir], contact_residues=contact_residues,
                          residues_map = None, contact_cutoff = cutoff, parallel = True, exacycle = exacycle, traj_top_structure = None, iterative=False,
                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes)

In [16]:
transform(existing_model = projection_operator_dir, features_directory = ref_features_dir, tica_dir = ref_tica_dir)


loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/phi_psi_chi2_allprot_tica_coords.h5"...
None
None
['/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/reference_receptors/4dkl_R_for_conformation.dataset', '/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/reference_receptors/5c1m.dataset']
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/reference_receptors/5c1m.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/reference_receptors/4dkl_R_for_conformation.dataset


In [25]:
import custom_tica
reload(custom_tica)
from custom_tica import *
transform_to_h5(projection_operator_dir, features_dir, projected_features_dir, worker_pool=None, parallel=True, normalizer=verboseload("%s/normalizer.h5" %features_dir))

loading "/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/normalizer.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/phi_psi_chi2_allprot_tica_coords.h5"...
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/phi_psi_chi2_allprot_projected.h5"... (<type 'list'>)
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_1.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_249.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_reseeded_rep_168.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_10.da

In [26]:
tica_object = verboseload(projection_operator_dir)

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/phi_psi_chi2_allprot_tica_coords.h5"...

loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_177.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_96.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_reseeded_rep_245.dataset


loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_178.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_97.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_reseeded_rep_246.dataset


loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5

In [31]:
tica_object.timescales_

array([ 249.59110712,  121.73650201,  105.44224943,   89.33989989,
         87.70752476,   66.19511558,   65.76380593,   47.0606975 ,
         44.4282818 ,   43.78414623])

In [ ]:
plot_columns(tica_dir, projected_features_dir, titles = ["tIC%d" %j for j in range(1,11)], tICA = True, scale = 1.0, refcoords_file = None)

In [32]:
tica_coords = verboseload(projected_features_dir)
pnas_coords = verboseload(user_defined_features_file)
tica_names = ["tIC.%d" %i for i in range(1,n_components+1)]
pnas_names = sorted(feature_name_residues_dict.keys())

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/phi_psi_chi2_allprot_projected.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/user_defined_features.h5"...


In [20]:
from plots import *

In [ ]:
plot_data_vs_data(np.concatenate(tica_coords), np.concatenate(pnas_coords), tica_names, pnas_names, analysis_dir)

In [33]:
tica_coords = verboseload(projected_features_dir)
np.shape(np.concatenate(tica_coords))

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/phi_psi_chi2_allprot_projected.h5"...


(890273, 10)

In [ ]:
f = load_file("/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_167.dataset")

In [ ]:
np.shape(f)

In [34]:
subsampled_features_dir = os.path.join(tica_dir, "subsampled_features")
if not os.path.exists(subsampled_features_dir): os.makedirs(subsampled_features_dir)

## 

In [9]:
tic_components_dir = tica_dir
important_contact_features = interpret_tIC_components(projection_operator_dir, tic_components_dir, feature_residues_pkl, n_tica_components=n_components, percentile=95)


Interpreting tIC 1
feature_importances_df.shape
(1, 7)
residue_importances_df.shape
(1, 3)
  feature_name   res_i   res_j  resid_i  resid_j  importance      feature
0  Asn109: Phi  Asn109  Asn109      109      109    1.000143  Asn109: phi
Using dark_background
       residue  importance  resid
Asn109  Asn109    1.000143    109
Using dark_background
Interpreting tIC 2
feature_importances_df.shape
(2, 7)
residue_importances_df.shape
(3, 3)
      feature_name   res_i   res_j  resid_i  resid_j  importance  \
1  Leu88 To Phe338   Leu88  Phe338       88      338    1.027029   
0      Asn109: Phi  Asn109  Asn109      109      109   -0.237379   

           feature  
1  LEU88 to PHE338  
0      Asn109: phi  
Using dark_background
       residue  importance  resid
Phe338  Phe338    1.027029    338
Leu88    Leu88    1.027029     88
Asn109  Asn109   -0.237379    109
Using dark_background
Interpreting tIC 3
feature_importances_df.shape
(4, 7)
residue_importances_df.shape
(4, 3)
           feature_

In [36]:
import interpret_tICs
reload(interpret_tICs)
from interpret_tICs import *
tic_subsampled_features_file = "%s/features_subsampled.pkl" % tica_dir
subsampled_features_dir = os.path.join(tica_dir, "subsampled_features")
if not os.path.exists(subsampled_features_dir): os.makedirs(subsampled_features_dir)
important_contact_features_pruned, important_contact_features_indices = find_non_zero_features(important_contact_features[0], feature_residues)
subsample_features(features_dir, important_contact_features_indices, important_contact_features_pruned, tic_subsampled_features_file, worker_pool=None)

loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_1.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_137.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_175.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_250.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_212.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_58.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_96.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_reseeded_rep_133.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/fe

In [ ]:
important_contact_features

In [37]:
protein_top_features = load_file(tic_subsampled_features_file)



loading /home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/features_subsampled.pkl




loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_119.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_157.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_195.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_4.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_232.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_78.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_reseeded_rep_

In [ ]:
protein_top_features[0].iloc[0]

In [ ]:
f = load_file("/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_1.dataset")
np.shape(f)

In [38]:
from ipyparallel import Client
rc = Client()
dview = rc[:]
dview.map(os.chdir, ['/home/enf/b2ar_analysis/conformation']*len(rc.ids))

print(len(rc.ids))


243
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/bu72_reimaged_rep_244.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/bu72_rep_127.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/bu72_rep_202.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/bu72_rep_240.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_reseeded_rep_93.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/bu72_reimaged_rep_130.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/bu72_rep_165.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/bu72_rep_48.dataset











loading /home/en

In [40]:
bu72_trajs = []
apo_trajs = []
oxy_trajs = []
suf_trajs = []
trv_trajs = []
trajs = get_trajectory_files(traj_dir, traj_ext)
#trajs = [t for t in get_trajectory_files(traj_dir, traj_ext) if "apo_reimaged" not in t and "oxy" not in t]
for i, traj in enumerate(trajs):
    if "bu72" in traj:
        bu72_trajs.append(i)
    elif "apo" in traj:
        apo_trajs.append(i)
    elif "oxy" in traj:
        oxy_trajs.append(i)
    elif "suf" in traj:
        suf_trajs.append(i)
    elif "trv" in traj:
        trv_trajs.append(i)

user_defined_coords = load_file(user_defined_features_file)

            

loading /home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/user_defined_features.h5
loading "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/user_defined_features.h5"...




loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/new_oxy_rep_276.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/new_oxy_rep_39.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/new_oxy_rep_85.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/new_oxy_rep_162.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/new_oxy_rep_47.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/new_oxy_rep_124.datasetloading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_unde

In [18]:
bu72_pnas = [user_defined_coords[i] for i in bu72_trajs]
apo_pnas = [user_defined_coords[i] for i in apo_trajs]
oxy_pnas = [user_defined_coords[i] for i in oxy_trajs]
suf_pnas = [user_defined_coords[i] for i in suf_trajs]

In [19]:
bu72_pnas_file = "%s/bu72.h5" % whole_trajectory_pnas
apo_pnas_file = "%s/apo.h5" % whole_trajectory_pnas
oxy_pnas_file = "%s/oxy.h5" % whole_trajectory_pnas
suf_pnas_file = "%s/suf.h5" % whole_trajectory_pnas

In [20]:
from imp import reload
import analysis
reload(analysis)
from analysis import *
import seaborn as sns
bu72_pnas_concatenated = np.concatenate(bu72_pnas)
print(np.shape(bu72_pnas_concatenated))
apo_pnas_concatenated = np.concatenate(apo_pnas)
oxy_pnas_concatenated = np.concatenate(oxy_pnas)
suf_pnas_concatenated = np.concatenate(suf_pnas)




verbosedump(bu72_pnas, bu72_pnas_file)
verbosedump(apo_pnas, apo_pnas_file)
verbosedump(oxy_pnas, oxy_pnas_file)
verbosedump(suf_pnas, suf_pnas_file)





columns=sorted(feature_name_residues_dict.keys())
#plot_columns(whole_trajectory_pnas, bu72_pnas_file, titles=columns, main = "BU72", tICA = False, scale = 1.0, refcoords_file = None)
#plot_columns(whole_trajectory_pnas, apo_pnas_file, titles=columns, main = "Apo", tICA = False, scale = 1.0, refcoords_file = None)

#df = pd.DataFrame(bu72_pnas_concatenated, columns=sorted(feature_name_residues_dict.keys()))
#sns.jointplot(x="tm6_tm3_dist", y="rmsd_npxxy_inactive", data=df, kind="kde")
#plt.hexbin(bu72_pnas_concatenated[:,0], bu72_pnas_concatenated[:,2], cmap=plt.cm.YlOrRd_r)

(245952, 10)
Saving "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/bu72.h5"... (<type 'list'>)
Saving "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/apo.h5"... (<type 'list'>)


In [21]:
oxy_tica_file = "%s/oxy_tica.h5" % tica_dir
bu72_tica_file = "%s/bu72_tica.h5" % tica_dir
apo_tica_file = "%s/apo_tica.h5" % tica_dir
suf_tica_file = "%s/suf_tica.h5" % tica_dir

In [22]:
tica_coords = verboseload(projected_features_dir)
oxy_tica = [tica_coords[i] for i in oxy_trajs]
bu72_tica = [tica_coords[i] for i in bu72_trajs]
apo_tica = [tica_coords[i] for i in apo_trajs]
suf_tica = [tica_coords[i] for i in suf_trajs]


verbosedump(oxy_tica, oxy_tica_file)
verbosedump(bu72_tica, bu72_tica_file)
verbosedump(apo_tica, apo_tica_file)
verbosedump(suf_tica, suf_tica_file)



columns = ["tIC%d" %i for i in range(1,n_components+1)]
#plot_columns(tica_dir, bu72_tica_file, titles=columns, main = "BU72", tICA = False, scale = 1.0, refcoords_file = None)
#plot_columns(tica_dir, apo_tica_file, titles=columns, main = "Apo", tICA = False, scale = 1.0, refcoords_file = None)




loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/phi_psi_chi2_allprot_projected.h5"...
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/oxy_tica.h5"... (<type 'list'>)
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_tica.h5"... (<type 'list'>)
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/apo_tica.h5"... (<type 'list'>)


In [23]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *
n_clusters = 1000
clusterer_dir = "%s/clusterer_%dclusters.h5" %(tica_dir, n_clusters)
#cluster_minikmeans(tica_dir, projected_features_dir, traj_dir, n_clusters=n_clusters, clusterer_dir=clusterer_dir)

In [24]:
print(clusterer_dir)

/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/clusterer_1000clusters.h5


In [25]:
oxy_clusterer_file = "%s/oxy_clusterer%d.h5" %(tica_dir, n_clusters)
bu72_clusterer_file = "%s/bu72_clusterer%d.h5" %(tica_dir, n_clusters)
apo_clusterer_file = "%s/apo_clusterer%d.h5" %(tica_dir, n_clusters)
suf_clusterer_file = "%s/suf_clusterer%d.h5" %(tica_dir, n_clusters)



In [26]:
clusterer = verboseload(clusterer_dir)
cluster_labels = clusterer.labels_
oxy_clusters = [cluster_labels[i] for i in oxy_trajs]
bu72_clusters = [cluster_labels[i] for i in bu72_trajs]
apo_clusters = [cluster_labels[i] for i in apo_trajs]

bu72_clusterer = copy.deepcopy(clusterer)
bu72_clusterer.labels_ = bu72_clusters
#if not os.path.exists(bu72_clusterer_file):
verbosedump(bu72_clusterer, bu72_clusterer_file)

apo_clusterer = copy.deepcopy(clusterer)
apo_clusterer.labels_ = apo_clusters
    
#if not os.path.exists(apo_clusterer_file):
verbosedump(apo_clusterer, apo_clusterer_file)

oxy_clusterer = copy.deepcopy(clusterer)
oxy_clusterer.labels_ = oxy_clusters
    
#if not os.path.exists(oxy_clusterer_file):
verbosedump(oxy_clusterer, oxy_clusterer_file)

suf_clusterer = copy.deepcopy(clusterer)
suf_clusterer.labels_ = suf_clusters
    
#if not os.path.exists(oxy_clusterer_file):
verbosedump(suf_clusterer, suf_clusterer_file)

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/clusterer_1000clusters.h5"...
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_clusterer1000.h5"... (<class 'msmbuilder.cluster.MiniBatchKMeans'>)
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/apo_clusterer1000.h5"... (<class 'msmbuilder.cluster.MiniBatchKMeans'>)
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/oxy_clusterer1000.h5"... (<class 'msmbuilder.cluster.MiniBatchKMeans'>)


In [27]:
import custom_msm
reload(custom_msm)
from custom_msm import *
from msm_resampled import *

In [29]:
apo_clusters_map = make_clusters_map(apo_clusterer)
bu72_clusters_map = make_clusters_map(bu72_clusterer)
oxy_clusters_map = make_clusters_map(oxy_clusterer)
suf_clusters_map = make_clusters_map(suf_clusterer)

600
204
230
623
3
0
414
278
112
570
0
740
162
498
76
68
492
455
1126
228
394
396
0
72
44
392
4
378
229
0
31
110
4
6
487
686
516
34
115
803
428
345
216
97
218
201
118
322
1
163
0
323
180
132
467
245
0
154
31
199
586
146
197
1066
377
0
464
405
0
331
754
325
73
564
730
821
20
741
174
687
38
18
0
288
100
225
199
907
164
607
665
0
532
7
0
66
321
571
233
360
0
191
104
0
4
0
12
386
609
564
298
458
556
4
267
356
40
77
148
513
322
122
210
27
2
257
1
166
223
156
626
505
510
587
561
2
6
276
50
324
388
23
190
0
356
11
182
28
9
12
254
211
616
294
0
126
148
138
66
0
771
9
17
443
127
120
246
519
458
558
0
0
175
113
435
0
42
177
119
57
57
57
442
0
0
261
323
511
78
122
119
186
135
442
31
179
316
0
136
267
323
245
74
17
488
595
0
739
302
184
1
75
522
231
166
537
379
9
238
0
719
633
133
37
33
91
539
379
0
217
164
161
4
0
370
156
0
45
428
767
181
60
56
402
25
265
176
160
36
532
90
4
137
395
491
210
164
0
59
344
193
667
67
534
150
29
79
470
436
42
464
0
372
0
846
0
255
254
0
288
1
157
366
0
0
544
0
0
13
0


In [54]:

plt.clf()
plot_timescales(apo_clusterer_file, n_clusters, tica_dir, main="Apo MSM Timescales", lag_times=list(range(1,101,10)))

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/apo_clusterer1000.h5"...
MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=1000, n_init=10, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)
MSM contains 6 strongly connected components above weight=1.00. Component 5 selected, with population 99.758362%
MSM contains 10 strongly connected components above weight=0.09. Component 9 selected, with population 99.700272%
MSM contains 15 strongly connected components above weight=0.05. Component 13 selected, with population 99.682608%
MSM contains 19 strongly connected components above weight=0.03. Component 15 selected, with population 99.567161%
MSM contains 22 strongly connected components above weight=0.02. Component 18 selected, with population 99.541675%
MSM 

In [55]:
plt.clf()
plot_timescales(bu72_clusterer_file, n_clusters, tica_dir, main="BU72 MSM Timescales", lag_times=list(range(1,101,10)))

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_clusterer1000.h5"...
MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=1000, n_init=10, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)
MSM contains 2 strongly connected components above weight=1.00. Component 0 selected, with population 100.000000%
MSM contains 5 strongly connected components above weight=0.09. Component 1 selected, with population 99.997919%
MSM contains 7 strongly connected components above weight=0.05. Component 2 selected, with population 99.997449%
MSM contains 10 strongly connected components above weight=0.03. Component 4 selected, with population 99.973489%
MSM contains 13 strongly connected components above weight=0.02. Component 6 selected, with population 99.962230%
MSM con

In [90]:
apo_lag_time=25
prior_counts = 0.0
apo_msm_dir = os.path.join(tica_dir, "apo_msm_lag-time%d_prior-counts%s_clusters%d.h5" %(apo_lag_time, str(prior_counts), n_clusters))
#if not os.path.exists(apo_msm_dir):
apo_msm = build_msm(apo_clusterer_file, apo_lag_time, apo_msm_dir, prior_counts)
#else:
#apo_msm = verboseload(apo_msm_dir)

import msm_resampled
reload(msm_resampled)
from msm_resampled import *
total_samples = 10000
num_trajs = len(apo_trajs)
prior_counts = 0.
apo_resampled_traj_to_frames_file = os.path.join(tica_dir, "apo_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(apo_lag_time, str(prior_counts), n_clusters, total_samples))
#if not os.path.exists(apo_resampled_traj_to_frames_file):
apo_resampled_traj_to_frames = resample_by_msm(total_samples, apo_msm, apo_clusters_map, num_trajs, apo_resampled_traj_to_frames_file)
#else:
apo_resampled_traj_to_frames = verboseload(apo_resampled_traj_to_frames_file)

apo_pnas = verboseload(apo_pnas_file)
apo_pnas_resampled_file = os.path.join(tica_dir, "apo_pnas_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(apo_lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(apo_pnas, apo_resampled_traj_to_frames, apo_pnas_resampled_file)
apo_pnas_resampled = verboseload(apo_pnas_resampled_file)

tica_concatenated = np.concatenate(verboseload(projected_features_dir))
tica_axes = [(np.min(tica_concatenated[:,i]), np.max(tica_concatenated[:,i])) for i in range(0, np.shape(tica_concatenated)[1])]

apo_tica = verboseload(apo_tica_file)
apo_tica_resampled_file = os.path.join(tica_dir, "apo_tica_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(apo_lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(apo_tica, apo_resampled_traj_to_frames, apo_tica_resampled_file)
apo_tica_resampled = verboseload(apo_tica_resampled_file)

apo_clusterer = verboseload(apo_clusterer_file)
apo_msm = verboseload(apo_msm_dir)

cluster_id_proportion_tuples = []

equilibrium_populations = apo_msm.populations_

for cluster_id in apo_msm.mapping_.keys():
    state_id = apo_msm.mapping_[cluster_id]
    cluster_id_proportion_tuples.append((cluster_id,equilibrium_populations[state_id]))
cluster_id_proportion_tuples = sorted(cluster_id_proportion_tuples, key=itemgetter(1), reverse=True)
    
plot_all_tics_and_clusters(tica_dir, apo_tica_resampled_file, apo_clusterer_file, lag_time, tic_range=range(0,n_components), main = "Apo_MSM_Reweighted", label = "cluster_id", active_cluster_ids = [c[0] for c in cluster_id_proportion_tuples[0:100]], intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=False, axes=tica_axes)

#plot_columns(tica_dir, apo_tica_resampled_file, titles = ["tIC.%d" %i for i in range(1,n_components+1)], main = "mOR_Apo_MSM_", tICA = False, scale = 1.0, refcoords_file = ref_tica_coords, axes=tica_axes, concatenate=False, reshape=False)

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/apo_clusterer1000.h5"...
fitting msm to trajectories with 1000 clusters and lag_time 25
MSM contains 16 strongly connected components above weight=0.04. Component 14 selected, with population 99.688272%
MarkovStateModel(ergodic_cutoff='on', lag_time=25, n_timescales=None,
         prior_counts=0.0, reversible_type='mle', sliding_window=True,
         verbose=True)
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/apo_msm_lag-time25_prior-counts0.0_clusters1000.h5"... (<class 'msmbuilder.msm.msm.MarkovStateModel'>)
fitted msm to trajectories with 894 states


In [ ]:
bu72_lag_time=25
prior_counts = 0.0
bu72_msm_dir = os.path.join(tica_dir, "bu72_msm_lag-time%d_prior-counts%s_clusters%d.h5" %(bu72_lag_time, str(prior_counts), n_clusters))
#if not os.path.exists(bu72_msm_dir):
bu72_msm = build_msm(bu72_clusterer_file, bu72_lag_time, bu72_msm_dir, prior_counts)
#else:
#bu72_msm = verboseload(bu72_msm_dir)

import msm_resampled
reload(msm_resampled)
from msm_resampled import *
total_samples = 10000
num_trajs = len(bu72_trajs)
prior_counts = 0.
bu72_resampled_traj_to_frames_file = os.path.join(tica_dir, "bu72_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(bu72_lag_time, str(prior_counts), n_clusters, total_samples))
#if not os.path.exists(bu72_resampled_traj_to_frames_file):
bu72_resampled_traj_to_frames = resample_by_msm(total_samples, bu72_msm, bu72_clusters_map, num_trajs, bu72_resampled_traj_to_frames_file)
#else:
bu72_resampled_traj_to_frames = verboseload(bu72_resampled_traj_to_frames_file)

bu72_pnas = verboseload(bu72_pnas_file)
bu72_pnas_resampled_file = os.path.join(tica_dir, "bu72_pnas_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(bu72_lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(bu72_pnas, bu72_resampled_traj_to_frames, bu72_pnas_resampled_file)
bu72_pnas_resampled = verboseload(bu72_pnas_resampled_file)

tica_concatenated = np.concatenate(verboseload(projected_features_dir))
tica_axes = [(np.min(tica_concatenated[:,i]), np.max(tica_concatenated[:,i])) for i in range(0, np.shape(tica_concatenated)[1])]

bu72_tica = verboseload(bu72_tica_file)
bu72_tica_resampled_file = os.path.join(tica_dir, "bu72_tica_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(bu72_lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(bu72_tica, bu72_resampled_traj_to_frames, bu72_tica_resampled_file)
bu72_tica_resampled = verboseload(bu72_tica_resampled_file)

bu72_clusterer = verboseload(bu72_clusterer_file)
bu72_msm = verboseload(bu72_msm_dir)

cluster_id_proportion_tuples = []

equilibrium_populations = bu72_msm.populations_

for cluster_id in bu72_msm.mapping_.keys():
    state_id = bu72_msm.mapping_[cluster_id]
    cluster_id_proportion_tuples.append((cluster_id,equilibrium_populations[state_id]))
cluster_id_proportion_tuples = sorted(cluster_id_proportion_tuples, key=itemgetter(1), reverse=True)
    
plot_all_tics_and_clusters(tica_dir, bu72_tica_resampled_file, bu72_clusterer_file, lag_time, tic_range=range(0,n_components), main = "bu72_MSM_Reweighted", label = "cluster_id", active_cluster_ids = [c[0] for c in cluster_id_proportion_tuples[0:100]], intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=False, axes=tica_axes)

#plot_columns(tica_dir, bu72_tica_resampled_file, titles = ["tIC.%d" %i for i in range(1,n_components+1)], main = "mOR_bu72_MSM_", tICA = False, scale = 1.0, refcoords_file = ref_tica_coords, axes=tica_axes, concatenate=False, reshape=False)

In [ ]:
oxy_lag_time=25
prior_counts = 0.0
oxy_msm_dir = os.path.join(tica_dir, "oxy_msm_lag-time%d_prior-counts%s_clusters%d.h5" %(oxy_lag_time, str(prior_counts), n_clusters))
#if not os.path.exists(oxy_msm_dir):
oxy_msm = build_msm(oxy_clusterer_file, oxy_lag_time, oxy_msm_dir, prior_counts)
#else:
#oxy_msm = verboseload(oxy_msm_dir)

import msm_resampled
reload(msm_resampled)
from msm_resampled import *
total_samples = 10000
num_trajs = len(oxy_trajs)
prior_counts = 0.
oxy_resampled_traj_to_frames_file = os.path.join(tica_dir, "oxy_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(oxy_lag_time, str(prior_counts), n_clusters, total_samples))
#if not os.path.exists(oxy_resampled_traj_to_frames_file):
oxy_resampled_traj_to_frames = resample_by_msm(total_samples, oxy_msm, oxy_clusters_map, num_trajs, oxy_resampled_traj_to_frames_file)
#else:
oxy_resampled_traj_to_frames = verboseload(oxy_resampled_traj_to_frames_file)

oxy_pnas = verboseload(oxy_pnas_file)
oxy_pnas_resampled_file = os.path.join(tica_dir, "oxy_pnas_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(oxy_lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(oxy_pnas, oxy_resampled_traj_to_frames, oxy_pnas_resampled_file)
oxy_pnas_resampled = verboseload(oxy_pnas_resampled_file)

tica_concatenated = np.concatenate(verboseload(projected_features_dir))
tica_axes = [(np.min(tica_concatenated[:,i]), np.max(tica_concatenated[:,i])) for i in range(0, np.shape(tica_concatenated)[1])]

oxy_tica = verboseload(oxy_tica_file)
oxy_tica_resampled_file = os.path.join(tica_dir, "oxy_tica_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(oxy_lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(oxy_tica, oxy_resampled_traj_to_frames, oxy_tica_resampled_file)
oxy_tica_resampled = verboseload(oxy_tica_resampled_file)

oxy_clusterer = verboseload(oxy_clusterer_file)
oxy_msm = verboseload(oxy_msm_dir)

cluster_id_proportion_tuples = []

equilibrium_populations = oxy_msm.populations_

for cluster_id in oxy_msm.mapping_.keys():
    state_id = oxy_msm.mapping_[cluster_id]
    cluster_id_proportion_tuples.append((cluster_id,equilibrium_populations[state_id]))
cluster_id_proportion_tuples = sorted(cluster_id_proportion_tuples, key=itemgetter(1), reverse=True)
    
plot_all_tics_and_clusters(tica_dir, oxy_tica_resampled_file, oxy_clusterer_file, lag_time, tic_range=range(0,n_components), main = "oxy_MSM_Reweighted", label = "cluster_id", active_cluster_ids = [c[0] for c in cluster_id_proportion_tuples[0:100]], intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=False, axes=tica_axes)

#plot_columns(tica_dir, oxy_tica_resampled_file, titles = ["tIC.%d" %i for i in range(1,n_components+1)], main = "mOR_oxy_MSM_", tICA = False, scale = 1.0, refcoords_file = ref_tica_coords, axes=tica_axes, concatenate=False, reshape=False)

In [ ]:
plot_columns(tica_dir, apo_pnas_resampled_file, titles = sorted(feature_name_residues_dict.keys()), main = "mOR_Apo_MSM_", tICA = False, scale = 1.0, refcoords_file = None, axes=None, concatenate=False, reshape=True)

In [106]:
bu72_lag_time=25
prior_counts = 0.0
bu72_msm_dir = os.path.join(tica_dir, "bu72_msm_lag-time%d_prior-counts%s_clusters%d.h5" %(bu72_lag_time, str(prior_counts), n_clusters))
if not os.path.exists(bu72_msm_dir):
    bu72_msm = build_msm(bu72_clusterer_file, bu72_lag_time, bu72_msm_dir, prior_counts)
else:
    bu72_msm = verboseload(bu72_msm_dir)
bu72_msm.timescales_

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_msm_lag-time25_prior-counts0.0_clusters1000.h5"...


array([  2.05861415e+05,   1.01633275e+05,   7.67499831e+03,
         4.46021556e+03,   3.64792534e+03,   2.25152524e+03,
         2.08751809e+03,   1.07625328e+03,   6.67398875e+02,
         6.27836408e+02,   4.76467608e+02,   3.76289275e+02,
         3.02133254e+02,   2.92930237e+02,   2.64425927e+02,
         2.59231441e+02,   2.37872457e+02,   2.37524328e+02,
         2.26435389e+02,   2.19769645e+02,   2.09408086e+02,
         1.69665877e+02,   1.63441320e+02,   1.50494187e+02,
         1.46523687e+02,   1.43502841e+02,   1.42593465e+02,
         1.39474757e+02,   1.37266268e+02,   1.36039189e+02,
         1.31378396e+02,   1.30309366e+02,   1.23728564e+02,
         1.22006106e+02,   1.12654333e+02,   1.10967076e+02,
         1.10475909e+02,   1.09923804e+02,   1.06261157e+02,
         1.03892623e+02,   9.81711030e+01,   9.61650870e+01,
         9.24662385e+01,   9.08718133e+01,   8.94584349e+01,
         8.72672774e+01,   8.67555463e+01,   8.63717344e+01,
         8.48014614e+01,

In [ ]:
fixed_assignments = dict()
for i, traj in enumerate(bu72_clusterer.labels_):
    fixed_assignments[i] = bu72_msm.transform(traj)

In [ ]:
fixed_assignments[0]

In [ ]:
print("hello")

In [ ]:
def pop_vs_counts(msm_obj, fixed_assignments, ax,**kwargs):

    pop_count_vect=[]
    for i in range(msm_obj.n_states_):
        pop_count_vect.append(np.sum([np.count_nonzero(fixed_assignments[j]==i)
                                      for j in fixed_assignments.keys()]))
    pop_count_vect = pop_count_vect/np.sum(pop_count_vect)
    ax.errorbar(np.log10(msm_object.populations_),np.log10(pop_count_vect),**kwargs)
    ax.plot([0,-6],[0,-6])
    ax.set_ylim([-6,0])
    ax.set_xlim([-6,0])
    ax.set_xlabel(" MSM Population",size=16)
    ax.set_ylabel("Counts Population",size=16)
    return pop_count_vect
f=figure()
ax=f.add_subplot(1,1,1)
pop_count_vect = pop_vs_counts(bu72_msm, fixed_assignments, ax fmt='o', color='r',label="RCSB-ASP")
ax.legend(loc='best',prop={'size':12})

In [107]:
total_samples = 10000
num_trajs = len(bu72_trajs)
bu72_resampled_traj_to_frames_file = os.path.join(tica_dir, "bu72_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(bu72_lag_time, str(prior_counts), n_clusters, total_samples))
if not os.path.exists(bu72_resampled_traj_to_frames_file):
    bu72_resampled_traj_to_frames = resample_by_msm(total_samples, bu72_msm, bu72_clusters_map, num_trajs, bu72_resampled_traj_to_frames_file)
else:
    bu72_resampled_traj_to_frames = verboseload(bu72_resampled_traj_to_frames_file)

Found number to sample per cluster based on equilibrium proporrtions.
Obtained random (trajectory, frame) pairs based on equilibrium populations
Rearranged equilibrium sampled frames based on trajectories
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"... (<type 'dict'>)


In [108]:
bu72_resampled_traj_to_frames_file

'/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5'

In [110]:
bu72_pnas = verboseload(bu72_pnas_file)
bu72_pnas_resampled_file = os.path.join(tica_dir, "bu72_pnas_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(bu72_lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(bu72_pnas, bu72_resampled_traj_to_frames, bu72_pnas_resampled_file)
bu72_pnas_resampled = verboseload(bu72_pnas_resampled_file)

loading "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/bu72.h5"...
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_pnas_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"... (<type 'numpy.ndarray'>)
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_pnas_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...


In [111]:
bu72_tica = verboseload(bu72_tica_file)
bu72_tica_resampled_file = os.path.join(tica_dir, "bu72_tica_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(bu72_lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(bu72_tica, bu72_resampled_traj_to_frames, bu72_tica_resampled_file)
bu72_tica_resampled = verboseload(bu72_tica_resampled_file)

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_tica.h5"...
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_tica_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"... (<type 'numpy.ndarray'>)
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_tica_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...


In [118]:
bu72_clusterer = verboseload(bu72_clusterer_file)
bu72_msm = verboseload(bu72_msm_dir)

cluster_id_proportion_tuples = []

equilibrium_populations = bu72_msm.populations_

for cluster_id in bu72_msm.mapping_.keys():
    state_id = bu72_msm.mapping_[cluster_id]
    cluster_id_proportion_tuples.append((cluster_id,equilibrium_populations[state_id]))
cluster_id_proportion_tuples = sorted(cluster_id_proportion_tuples, key=itemgetter(1), reverse=True)
    
plot_all_tics_and_clusters(tica_dir, bu72_tica_resampled_file, bu72_clusterer_file, lag_time, tic_range=range(0,10), main = "bu72_MSM_Reweighted", label = "cluster_id", active_cluster_ids = [c[0] for c in cluster_id_proportion_tuples[0:100]], intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=False, axes=tica_axes)

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_clusterer1000.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_msm_lag-time25_prior-counts0.0_clusters1000.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_tica_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_clusterer1000.h5"...
[[-0.6766631  -0.06976278 -0.209463   ..., -0.43445881 -0.04407875
  -0.34019803]
 [ 1.64564213 -0.08121973 -0.12899196 ..., -0.35194832 -0.49647686
  -0.45887836]
 [-0.68615465 -

In [119]:
bu72_pnas_file

'/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/bu72.h5'

In [120]:
analysis_dir

'/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/analysis_n_clusters1000_random'

In [121]:
tica_resampled = np.vstack([verboseload(bu72_pnas_resampled_file)[::1,], verboseload(apo_pnas_resampled_file)[::1,]])
mins = np.min(tica_resampled, axis=0)
maxs = np.max(tica_resampled, axis=0)
stds = np.std(tica_resampled, axis=0)
custom_lims = [[mins[i] - 1.*stds[i], maxs[i] + 1.*stds[i]] for i in range(0,len(mins))]

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_pnas_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/apo_pnas_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...


In [122]:
ref_pnas = load_file("/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5")
ref_pnas = np.hstack([np.array(x).T for x in ref_pnas])

loading /home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5
loading "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5"...


In [123]:
def get_features(features_dir):
    feature_files = get_trajectory_files(features_dir, ".dataset")
    features = np.vstack([load_file(f) for f in feature_files])
    print(features.shape)
    return features

ref_pp_features = pd.DataFrame(get_features(ref_features_dir), columns=[str(tuple(f)) for f in feature_residues])

ref_pp_tica = pd.read_csv(ref_tica_coords, header=None).values
#ref_pp_tica.columns = ["mOR tIC.%d" %i for i in range(1,ref_pp_tica.shape[1])]

ref_pnas = load_file("/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5")
ref_pnas = np.hstack([np.array(x).T for x in ref_pnas])

#ref_pnas = pd.DataFrame(ref_pnas, columns=sorted(feature_name_residues_dict.keys()))

#ref_features = pd.concat([ref_pp_features*10., ref_lp_features*10., ref_pp_tica, ref_lp_tica, ref_pnas], axis=1)

loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/reference_receptors/4dkl_R_for_conformation.dataset
loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/reference_receptors/5c1m.dataset
(2, 4440)
loading /home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5
loading "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5"...


In [124]:
ref_pp_tica

array([[ 1.17404121,  0.09484967,  0.0859546 ,  0.23719442,  0.05820045,
         0.06633734,  1.22745619,  0.13890614,  0.37880338,  0.04773916],
       [ 0.48768246,  0.09465974,  0.09523272,  0.23220757,  0.36926021,
         0.10936899,  1.06149709,  0.17976553,  0.49342095,  0.12489334]])

In [125]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *
jointplots(bu72_pnas_resampled[::1,:], analysis_dir, titles = sorted(feature_name_residues_dict.keys()), main = "mOR_BU72_MSM_", refcoords = ref_pnas, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=20, min_density=None, max_diff=3., custom_lims=custom_lims, n_levels=15, n_pts=200j, parallel=True)

Making delta G plots.
Done plotting columns
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
min_density=0.0min_density=0.0min_density=0.0min_density=0.0min_density=0.0min_density=0.0





Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)





[[  7.25469112   3.95825744]
 [ 11.95560455   0.        ]][[ 3.95825744  0.        ]
 [ 0.          3.9582572 ]][[ 0.          0.64148617]
 [ 3.9582572   0.        ]][[ 0.64148617  0.        ]
 [ 0.          0.64148796]][[ 0.          2.36107397]
 [ 0.64148796  0.        ]][[ 2.36107397  0.        ]
 [ 0.          2.36107469]]





(2, 9990)(


(-0.60000000000000009, 3.0)[[  7.25469112   2.36107397]
 [ 11.95560455   0.        ]][[ 3.95825744  0.        ]
 [ 0.          2.36107469]][[  0.           6.46793795]
 [  3.9582572   12.95613289]][[  0.64148617   9.15980816]
 [  0.          13.01761985]][[  0.           8.57000589]
 [  0.64148796  11.52658105]]





[[  0.           6.46793795]
 [  2.36107469  12.95613289]](2, 9990)(2, 9990)(2, 9990)(2, 9990)(2, 9990)





(2, 9990)Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE

Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.min_density=0.0Computed 2D KDE





Computed Density. Now plotting.(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)Computed Density. Now plotting.





(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)[[  7.25469112   0.        ]
 [ 11.95560455   2.36107469]][[  3.95825744   6.46793795]
 [  

In [126]:
ref_pnas

array([[  7.25469112,   3.95825744,   0.        ,   0.64148617,
          0.        ,   2.36107397,   0.        ,   6.46793795,
          9.15980816,   8.57000589],
       [ 11.95560455,   0.        ,   3.9582572 ,   0.        ,
          0.64148796,   0.        ,   2.36107469,  12.95613289,
         13.01761985,  11.52658105]])

In [127]:
jointplots(apo_pnas_resampled[::1,:], analysis_dir, titles = sorted(feature_name_residues_dict.keys()), main = "mOR_APO_MSM_", refcoords=ref_pnas, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=20, min_density=None, max_diff=3., custom_lims=custom_lims, n_levels=10)

Making delta G plots.
Done plotting columns
(2, 9981)
(2, 9981)
(2, 9981)
(2, 9981)
(2, 9981)
(2, 9981)
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
min_density=0.0min_density=0.0min_density=0.0min_density=0.0min_density=0.0min_density=0.0





Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)





[[  7.25469112   3.95825744]
 [ 11.95560455   0.        ]][[ 3.95825744  0.        ]
 [ 0.          3.9582572 ]][[ 0.          0.64148617]
 [ 3.9582572   0.        ]][[ 0.64148617  0.        ]
 [ 0.          0.64148796]][[ 0.          2.36107397]
 [ 0.64148796  0.        ]][[ 2.36107397  0.        ]
 [ 0.          2.36107469]]





(2, 9981)(


(-0.60000000000000009, 3.0)[[  7.25469112   2.36107397]
 [ 11.95560455   0.        ]][[ 3.95825744  0.        ]
 [ 0.          2.36107469]][[  0.           6.46793795]
 [  3.9582572   12.95613289]][[  0.64148617   9.15980816]
 [  0.          13.01761985]][[  0.           8.57000589]
 [  0.64148796  11.52658105]]





[[  0.           6.46793795]
 [  2.36107469  12.95613289]](2, 9981)(2, 9981)(2, 9981)(2, 9981)(2, 9981)





(2, 9981)Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE

Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.min_density=0.0Computed 2D KDE





Computed Density. Now plotting.(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)Computed Density. Now plotting.





(-0.60000000000000009, 3.0)(-0.60000000000000009, 3.0)[[  7.25469112   0.        ]
 [ 11.95560455   2.36107469]][[  3.95825744   6.46793795]
 [  

In [128]:
tica_resampled = np.vstack([verboseload(bu72_tica_resampled_file)[::1,], verboseload(apo_tica_resampled_file)[::1,]])
mins = np.min(tica_resampled, axis=0)
maxs = np.max(tica_resampled, axis=0)
stds = np.std(tica_resampled, axis=0)
custom_lims = [[mins[i] - 1.*stds[i], maxs[i] + 1.*stds[i]] for i in range(0,len(mins))]


loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_tica_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/apo_tica_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...


In [129]:
print(custom_lims)
print(custom_lims[3][1])

[[-1.6815236129225455, 3.9936155588298856], [-0.93436979653911933, 5.3285590654389789], [-2.0872627348017612, 10.731372972939612], [-1.7215703683596415, 11.10486134725757], [-5.5817516049567857, 8.4206709522359091], [-6.1251206594326328, 11.080091779676984], [-4.3750060440664686, 4.6032346351836022], [-2.6710331478047955, 4.6297940466000611], [-10.129626249154446, 3.5089563887161215], [-5.6786383102719338, 6.4384059688313666]]
11.1048613473


In [130]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *
refcoords_jointplots_dir = "%s/refcoords_jointplots" %analysis_dir
if not os.path.exists(refcoords_jointplots_dir):
    os.makedirs(refcoords_jointplots_dir)
jointplots(verboseload(bu72_tica_resampled_file)[::1,:], refcoords_jointplots_dir, titles = ["tIC%d" %j for j in range(1,n_components+1)], main = "BU72 MSM", refcoords=ref_pp_tica, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=10, min_density=None, max_diff=5., custom_lims=custom_lims, n_pts=200j, n_levels=15)

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_tica_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...
Making delta G plots.
Done plotting columns
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
min_density=0.0min_density=0.0min_density=0.0min_density=0.0min_density=0.0min_density=0.0





Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)





[[ 1.17404121  0.09484967]
 [ 0.48768246  0.09465974]][[ 0.09484967  0.0859546 ]
 [ 0.09465974  0.09523272]][[ 0.0859546   0.23719442]
 [ 0.09523272  0.23220757]][[ 0.23719442  0.05820045]
 [ 0.23220757  


(-1.0, 5.0)[[ 1.17404121  0.06633734]
 [ 0.48768246  0.10936899]][[ 0.09484967  1.22745619]
 [ 0.09465974  1.06149709]][[ 0.0859546   0.13890614]
 [ 0.09523272  0.17976553]][[ 0.23719442  0.37880338]
 [ 0.23220757  0.49342095]][[ 0.05820045  0.04773916]
 [ 0.36926021  0.12489334]]





[[ 1.22745619  0.13890614]
 [ 1.06149709  0.17976553]](2, 9990)(2, 9990)(2, 9990)(2, 9990)(2, 9990)





(2, 9990)Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE

Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.min_density=0.0Computed 2D KDE





Computed Density. Now plotting.(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)Computed Density. Now plotting.





(-1.0, 5.0)(-1.0, 5.0)[[ 1.17404121  1.22745619]
 [ 0.48768246  1.06149709]][[ 0.09484967  0.13890614]
 [ 0.09465974  0.17976553]][[ 0.0859546   0.37880338]
 [ 0.09523272  0.49342095]][[ 0.23719442  0.04773916]
 [ 0.23220757  0.12489334]]





[

In [131]:
np.concatenate(bu72_tica).shape

(245952, 10)

In [132]:
np.concatenate(apo_tica).shape

(233505, 10)

In [136]:
jointplots(verboseload(apo_tica_resampled_file)[::1,:], refcoords_jointplots_dir, titles = ["tIC%d" %j for j in range(1,n_components+1)], main = "Apo MSM", refcoords=ref_pp_tica, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=10, min_density=None, max_diff=5., custom_lims=custom_lims, n_pts=200j, n_levels=15, parallel=False)

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/apo_tica_msm_lag-time25_prior-counts0.0_clusters1000_resampled.h5"...
Making delta G plots.
(2, 9981)
Computed 2D KDE
min_density=0.0
Computed Density. Now plotting.
(-1.0, 5.0)
[[ 1.17404121  0.09484967]
 [ 0.48768246  0.09465974]]
(2, 9981)
Computed 2D KDE
Computed Density. Now plotting.
(-1.0, 5.0)
[[ 1.17404121  0.0859546 ]
 [ 0.48768246  0.09523272]]
(2, 9981)
Computed 2D KDE


KeyboardInterrupt: 

In [138]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *
jointplots(verboseload(bu72_tica_resampled_file)[::1,:], analysis_dir, titles = ["tIC%d" %j for j in range(1,n_components+1)], main = "BU72 MSM", refcoords=None, axes=None, data_j=bu72_pnas_resampled, titles_j=sorted(feature_name_residues_dict.keys()), reshape=False, max_tIC=10, min_density=None, max_diff=5., custom_lims=None)

loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_tica_msm_lag-time25_prior-counts0.0_clusters1000_resampled_10000.h5"...
Making delta G plots.
Done plotting columns
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
(2, 9990)
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
min_density=0.0min_density=0.0min_density=0.0min_density=0.0min_density=0.0min_density=0.0





Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
Computed Density. Now plotting.
(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)





(2, 9990)(2, 9990)(2, 9990)(2, 9990)(2, 9990)(2, 9990)





Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed Density. Now plotting.Computed Density.

(2, 9990)(2, 9990)





Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.





(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)





(2, 9990)(2, 9990)(2, 9990)(2, 9990)(2, 9990)(2, 9990)





Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.Computed Density. Now plotting.





(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)(-1.0, 5.0)





(2, 9990)(2, 9990)(2, 9990)(2, 9990)(2, 9990)(2, 9990)





Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed 2D KDE
Computed Density. Now plotting.Computed 

In [ ]:
import detect_intermediates
reload(detect_intermediates)
from detect_intermediates import *


In [ ]:
#sample_tIC_regions_silverman(np.concatenate(verboseload(projected_features_dir))[::100,:], analysis_dir)
#get_kde_mins_and_maxes(np.concatenate(verboseload(projected_features_dir))[::100,:], analysis_dir)
sample_kde_maxima(verboseload(projected_features_dir), analysis_dir, trajs)

In [ ]:
jointplots(verboseload(apo_tica_resampled_file)[::1,:], analysis_dir, titles = ["tIC%d" %j for j in range(1,n_components+1)], main = "Apo MSM", refcoords_file = None, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=5, min_density=None, max_diff=5., custom_lims=custom_lims)

In [ ]:
plot_columns(tica_dir, bu72_pnas_resampled_file, titles = sorted(feature_name_residues_dict.keys()), main = "mOR_BU72_MSM_", tICA = False, scale = 1.0, refcoords_file = None, axes=None, concatenate=False, reshape=True)

In [ ]:
#bu72_tica = verboseload(bu72_tica_file)
#bu72_tica_resampled_file = os.path.join(tica_dir, "bu72_tica_msm_lag-time%d_prior-counts%s_clusters%d_resampled.h5" %(lag_time, str(prior_counts), n_clusters))
#resample_features_by_msm_equilibirum_pop(bu72_tica, bu72_resampled_traj_to_frames, bu72_tica_resampled_file)
#bu72_tica_resampled = verboseload(bu72_tica_resampled_file)
plot_columns(tica_dir, bu72_tica_resampled_file, titles = ["tIC.%d" %i for i in range(1,n_components+1)], main = "mOR_BU72_MSM_", tICA = False, scale = 1.0, refcoords_file = ref_tica_coords, axes=tica_axes, concatenate=False, reshape=True)

In [ ]:
bu72_clusterer = verboseload(bu72_clusterer_file)
bu72_msm = verboseload(bu72_msm_dir)

cluster_id_proportion_tuples = []

equilibrium_populations = bu72_msm.populations_

for cluster_id in bu72_msm.mapping_.keys():
    state_id = bu72_msm.mapping_[cluster_id]
    cluster_id_proportion_tuples.append((cluster_id,equilibrium_populations[state_id]))
cluster_id_proportion_tuples = sorted(cluster_id_proportion_tuples, key=itemgetter(1), reverse=True)

#plot_all_tics_and_clusters(tica_dir, bu72_tica_resampled_file, bu72_clusterer_file, lag_time, tic_range=[0,3], main = "BU72_MSM_Reweighted", label = "cluster_id", active_cluster_ids = [c[0] for c in cluster_id_proportion_tuples[0:100]], intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=False, axes=tica_axes)


In [140]:
n_samples = 5
samples_dir = "%s/clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
print(samples_dir)
    
sample_from_clusterer(bu72_clusterer_file, projected_features_dir, trajs, 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=dview, parallel=False)

/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/clusterer_1000clusters_5samples_samples_kdtree
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_clusterer1000.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/phi_psi_chi2_allprot_projected.h5"...
loading "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/bu72_clusterer1000.h5"...
Saving "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t5_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt05/clusterer_1000clusters_5samples_samples_kdtree_indices.h5"... (<type 'numpy.nd

In [ ]:
n_samples = 5
samples_dir = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)

sample_from_clusterer(apo_clusterer_file, projected_features_dir, trajs, 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=dview, parallel=False)

In [ ]:
bu72_tica_resampled_file

In [ ]:
feature_name_residues_dict.keys()

In [ ]:

np.shape(apo_pnas_resampled)
msm_dir = os.path.join(tica_dir, "msm_lag-time%d_prior-counts%s_clusters%d_analysis" %(lag_time, str(prior_counts), n_clusters))
if not os.path.exists(msm_dir): os.makedirs(msm_dir)

#plot_columns(msm_dir, apo_pnas_resampled_file, titles = feature_name_residues_dict.keys(), main = "Apo-mOR", tICA = False, scale = 1.0, refcoords_file = None, axes=None, concatenate=False)

In [ ]:
import pip

def install(package):
    pip.main(['install', package])
    
install("moviepy")

In [ ]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *

for i, traj in enumerate(get_trajectory_files(traj_dir, traj_ext)):
    if "apo_rep_68." in traj:
        index=i
        break

jointplot(apo_pnas_resampled[::10,[6,3]], "%s/test.pdf" %msm_dir, trajectory=apo_pnas[index][::1,[6,3]], 
          titles=["TM6 to TM3 distance", "RMSD of NPxxY to Inactive"], main = "mOR Activation Pathway", 
          video_file = "%s/apo_68.mp4" %msm_dir, custom_xlim=[4.0, 16.0], custom_ylim= [0.0, 1.0])
#test("%s/test2.mp4" %msm_dir)

In [ ]:
build_msm(bu72_clusterer_file, lag_time, msm_model_dir)
build_msm(apo_clusterer_file, lag_time, msm_model_dir)

In [ ]:
tics_1_3_file = os.path.join(tica_dir, "tICs_1_3.h5")
tics = load_file(projected_features_dir)
tics_1_3 = [x[:,(0,2)] for x in tics]
verbosedump(tics_1_3, tics_1_3_file)

clusterer16_clusters = os.path.join(tica_dir, "clusterer16_clusters_tICs1_3.h5")
cluster_minikmeans(tica_dir, projected_features_dir, traj_dir, n_clusters=16, clusterer_dir=clusterer16_clusters,tICs=[1,3])

In [ ]:
closest_indices = find_closest_indices_to_cluster_center(tics_1_3_file, clusterer16_clusters)

In [ ]:
closest_indices

In [ ]:
from scipy.stats import gamma 
x = np.linspace(0.,16.0, 100)
y = gamma.pdf(x, 5.0, loc=8.0, scale=1)
plt.plot(x,y)

In [ ]:
from scipy.stats import gamma 
x = np.linspace(0.,16.0, 100)
y = gamma.pdf(x, 3.0, loc=8.0, scale=0.4)
plt.plot(x,y)
plt.show()

In [ ]:
reseed_from_clusterer(oxy_clusterer_file, "oxy", tica_dir, oxy_tica_file, [trajs[i] for i in oxy_trajs])

In [ ]:
min_pop = [3, 4, 6, 8, 9, 26, 29, 30, 31, 32, 33, 39, 41, 43, 52, 55]


In [ ]:
plot_all_tics_and_clusters(tica_dir, oxy_tica_file, oxy_clusterer_file, lag_time, tic_range=range(0,5), main = "Oxy_Reseeds", label = "cluster_id", active_cluster_ids = min_pop, intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=True, axes=tica_axes)